# Machine Learning NLP notebook for idenitfying ML methods papers in life science jorunal 
## 1. 

In [15]:
# 1. Preprocess DOME data & get insights from the literature data for use with developing ML lit triage model 

# 1A. DOME abstract and title

# 1. Import Necessary Libraries  
import pandas as pd
import numpy as np
import re
import nltk
import sklearn
from os import listdir
from os.path import isfile, join
# import spacy - depedncy issues avoid for now 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# 2. Load Data
# Load the DOME abstract and title data
os.lisir('.')

# Ensures that all required NLP libraries (NLTK, SpaCy, Scikit-learn, etc.) are available.  

# 2. Text Cleaning & Normalization  
# ✔ Lowercasing (Step 1.1) → Enables case-insensitive matching.  
# ✔ Text Normalization (Step 1.8, moved earlier) → Expands contractions, converts numbers, standardizes abbreviations.  

# 3. Tokenization & Basic Cleaning  
# ✔ Tokenization (Step 1.2) → Splits text into individual words or subwords.  
# ✔ Removing Punctuation (Step 1.3) → Ensures proper word separation.  
# ✔ Removing Stopwords (Step 1.4) → Eliminates commonly occurring but uninformative words.  

# 4. Lemmatization & Stemming  
# ✔ Lemmatization OR Stemming (Step 1.5) → Converts words to their root form.  
# (Lemmatization is preferable for accuracy; stemming is faster but less precise.)  

# 5. Feature Extraction (Reorganized for clarity)  
# ✔ Part-of-Speech (POS) Tagging & Counts (Step 1.6.1)  
# ✔ Named Entity Recognition (NER) (Step 1.6.2)  
# ✔ Term Frequency - Inverse Document Frequency (TF-IDF) (Step 1.6.3)  
# ✔ Word Embeddings (Word2Vec, GloVe, BERT, etc.) (Step 1.6.4 - remove “FREQUENCIES” since embeddings are dense vectors, not simple word counts.)  

# 6. Vectorization (Final Step)  
# ✔ TF-IDF OR Embeddings (Step 1.9) → Converts text into a numerical representation suitable for ML models.  
# (Vectorization is technically part of feature extraction, so this step can be merged with 1.6 if preferred.)


# 1B. DOME full text 


NameError: name 'os' is not defined

In [ ]:
# Identify using ml ontology and others rleevant ML words not within - eg: model types etc


In [ ]:
# 2. Determine if wider corpus of ML papers needed - automatically find some papers and then also preprocess
# could dtermine using text word mining
#random papers form lit suggest or negatiev search of terms - eg noo model/ml etc 
#  

In [ ]:
# 3. Preprocess all ML papers

In [ ]:
# 4. Download all papers mentioning machine learning and AI from EPMC 

In [ ]:
# 5. Deploy ML model to predict if a paper is about ML or not

In [ ]:
# 6. Analyse top papers and journals insights from the literature data